In [1]:
from datasets import load_dataset
dataset = load_dataset("csv", data_files="train.csv", split='train')

dataset

Using custom data configuration default-eecc797671eae6bc
Found cached dataset csv (/Users/olang/.cache/huggingface/datasets/csv/default-eecc797671eae6bc/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


Dataset({
    features: ['S1', 'S2', 'label'],
    num_rows: 6000
})

In [3]:
print(dataset[0])

{'S1': 'Scholar Ranking 2023: Ranking of Computer Science Departments Based on Faculty Citations', 'S2': 'Evident: a Development Methodology and a Knowledge Base Topology for Data Mining, Machine Learning and General Knowledge Management', 'label': 2}


In [5]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [7]:

all_cols = ['label']

for part in ['S1', 'S2']:               # S1 = premise, S2 = hypothesis
    dataset = dataset.map(
        lambda x: tokenizer(
            x[part], max_length=128, padding='max_length',
            truncation=True
        ), batched=True
    )
    for col in ['input_ids', 'attention_mask']:
        dataset = dataset.rename_column(
            col, part+'_'+col
        )
        all_cols.append(part+'_'+col)
print(all_cols)

Loading cached processed dataset at /Users/olang/.cache/huggingface/datasets/csv/default-eecc797671eae6bc/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-bbffd7ff4ac0f9ad.arrow
Loading cached processed dataset at /Users/olang/.cache/huggingface/datasets/csv/default-eecc797671eae6bc/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-29ce0419069efcaa.arrow


['label', 'S1_input_ids', 'S1_attention_mask', 'S2_input_ids', 'S2_attention_mask']


In [8]:
import torch

# covert dataset features to PyTorch tensors
dataset.set_format(type='torch', columns=all_cols)

# initialize the dataloader
batch_size = 16
loader = torch.utils.data.DataLoader(
    dataset, batch_size=batch_size, shuffle=True
)

In [9]:
# define mean pooling function
def mean_pool(token_embeds, attention_mask):
    # reshape attention_mask to cover 768-dimension embeddings
    in_mask = attention_mask.unsqueeze(-1).expand(
        token_embeds.size()
    ).float()
    # perform mean-pooling but exclude padding tokens (specified by in_mask)
    pool = torch.sum(token_embeds * in_mask, 1) / torch.clamp(
        in_mask.sum(1), min=1e-9
    )
    return pool

In [10]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'

ffnn = torch.nn.Linear(768*3, 3)
loss_func = torch.nn.CrossEntropyLoss()
 
#  move layers to device
ffnn.to(device)
loss_func.to(device)

CrossEntropyLoss()

In [13]:
from transformers.optimization import get_linear_schedule_with_warmup
from transformers import AdamW

# we would initialize everything first
optim = AdamW(model.parameters(), lr=1e-5)

# num epochs
epochs = 1

# and setup a warmup for the first ~10% steps
total_steps = int(len(dataset) / batch_size)
warmup_steps = int(0.1 *total_steps * epochs)
scheduler = get_linear_schedule_with_warmup(
		optim, num_warmup_steps=warmup_steps,
  	num_training_steps=total_steps - warmup_steps
)
	
# then during the training loop we update the scheduler per step
# scheduler.step()

NameError: name 'model' is not defined